In [2]:
import csv
import json
import pprint
import gzip
from collections import defaultdict
import string


import time
import calendar

pathAmazon = "C:/Users/Ian/Documents/PythonDataProducts4PredictiveAnalytics/BasicDataProcessing&Visualization/Final_Course1/datasets/amazon_reviews_us_Gift_Card_v1_00.tsv.gz"

In [4]:
fileAmazon = gzip.open(pathAmazon, 'rt', encoding="utf8")
dataAmazon = csv.reader(fileAmazon, delimiter='\t') 
header = next(dataAmazon)

In [5]:
datasetAmazon = []
for line in dataAmazon:
    # Construimos el diccionario dict(zip(clave, valor))
    d = dict(zip(header, line))
    for field in ['helpful_votes', 'star_rating', 'total_votes']:
        d[field] = int(d[field])
    for field in ['verified_purchase', 'vine']:
        if(d[field] == 'Y'):
            d[field] = True
        else:
            d[field] = False
    datasetAmazon.append(d)

La estructura de nuestros datos se ve así:

In [6]:
print("Total de datos: ", len(datasetAmazon))
pprint.pprint(datasetAmazon[0])

Total de datos:  148310
{'customer_id': '24371595',
 'helpful_votes': 0,
 'marketplace': 'US',
 'product_category': 'Gift Card',
 'product_id': 'B004LLIL5A',
 'product_parent': '346014806',
 'product_title': 'Amazon eGift Card - Celebrate',
 'review_body': 'Great birthday gift for a young adult.',
 'review_date': '2015-08-31',
 'review_headline': 'Five Stars',
 'review_id': 'R27ZP1F1CD0C3Y',
 'star_rating': 5,
 'total_votes': 0,
 'verified_purchase': True,
 'vine': False}


# Filtrado de datos  
Se trabajara los datos en formato de diccionario para una mejor manipulación de los datos 

### Filtrado por fecha
Para esto, vamos a convertir el campo review_date en integer, tomar solo el año y crear un nuevo campo llamado _yearInt_ para filtrar los datos más recientes.  
**Para esto, devemos de realizar un preprocesamiento ya que algunos documentos no tienen ningún valor en ese campo**

In [7]:
cleanDSA = [d for d in datasetAmazon if 'review_date' in d]
print(len(cleanDSA))

148309


In [10]:
for d in cleanDSA:
    d['yearInt'] = int(d['review_date'][:4])

Limpio nuestro conjunto de datos, vamos a realizar el filtrado de opiniones que realizadas a partir del 2010

In [12]:
cleanDSA = [d for d in cleanDSA if d['yearInt'] > 2009]
print(len(cleanDSA))

148095


### Filtramos las opiniones con mayor relevancia.  
Para esto consideramos dos parámetros:  
* Tiene menos de tres votos de utilidad. Si tienen pocos votos, la escencia de la relevancia no es muy acertada
* Su relación de utilidad es mayor que 0.5. Su radio de ayuda es mayor a 0.5

In [13]:
cleanDSA = [d for d in cleanDSA if d['total_votes'] < 3  or d['helpful_votes']/d['total_votes'] >= 0.5]
print(len(cleanDSA))

147801


### Filtramos por actividad de usuario.  
Para esto, realizamos un conteo de las apariciones de los usuarios que más comentarios tienen

In [14]:
nReviewsperUser = defaultdict(int)
for d in cleanDSA:
    nReviewsperUser[d['customer_id']] += 1

### Filtramos a los usuarios que tengan como mínimo dos reseñas  
Para esto, realizamos el conteo de número de apariciones en el conjunto de datos

In [17]:
cleanDSA= [d for d in cleanDSA if nReviewsperUser[d['customer_id']] >= 2]
print(len(cleanDSA))

11172


### Filtramos por longitud de la reseña
Realizamos un filtro de reseñas que son muy cortas (10 palabras o menos) y para esto, separamos las palabras y realizamos un conteo

In [18]:
cleanDSA = [d for d in cleanDSA if len(d['review_body'].split()) >= 10]
print(len(cleanDSA))

7033


In [21]:
string.punctuation

'!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~'

# Manejo de fechas  
Tenemos una fecha de muestra legible para los humanos para trabajar con los métodos para la transformación de fechas.

### Trabajaremos con una fecha inicial en formato string

In [4]:
timeString = "2018-07-26 01:36:02"

La función _time.strptime()_ recibe dos argumentos:  
* El primero es la cadena donde se encuentra la fecha
* El segundo argumento es una cadena con el formato de la fecha del string del primer parámetro  
  
Esto nos devuelve una estructura de tiempo donde identifica año, mes, día, hora, minuto y segundo así como el día de la semana y el dia del año.  
**Esta función es ideal cuando queremos extraer características de los datos**

In [5]:
timeStruct = time.strptime(timeString, "%Y-%m-%d %H:%M:%S")
timeStruct

time.struct_time(tm_year=2018, tm_mon=7, tm_mday=26, tm_hour=1, tm_min=36, tm_sec=2, tm_wday=3, tm_yday=207, tm_isdst=-1)

In [6]:
help(time.strptime)

Help on built-in function strptime in module time:

strptime(...)
    strptime(string, format) -> struct_time
    
    Parse a string to a time tuple according to a format specification.
    See the library reference manual for formatting codes (same as
    strftime()).
    
    Commonly used format codes:
    
    %Y  Year with century as a decimal number.
    %m  Month as a decimal number [01,12].
    %d  Day of the month as a decimal number [01,31].
    %H  Hour (24-hour clock) as a decimal number [00,23].
    %M  Minute as a decimal number [00,59].
    %S  Second as a decimal number [00,61].
    %z  Time zone offset from UTC.
    %a  Locale's abbreviated weekday name.
    %A  Locale's full weekday name.
    %b  Locale's abbreviated month name.
    %B  Locale's full month name.
    %c  Locale's appropriate date and time representation.
    %I  Hour (12-hour clock) as a decimal number [01,12].
    %p  Locale's equivalent of either AM or PM.
    
    Other codes may be available on yo

### Trabajaremos ahora con la fecha como estructura de tiempo de Python  
Las convertiremos a formato de número con las funciones _calendar.timegm(timeStruct)_ y _time.mktime(timeStrict)_ las cuales reciben como argumento una estructura de tiempo. Esto con el fin de poder manejar operaciones con las fechas

_timgm()_ asume la estructura del tiempo con la hora UTC

In [17]:
t1 = calendar.timegm(timeStruct)

_mktime()_ asuma que la estructura en una hora local

In [18]:
t2 = time.mktime(timeStruct)

In [19]:
print(t1)
print(t2)

1532568962
1532586962.0


In [20]:
t1 + 60 * 60 * 24 * 5

1532568962

### Conversión de tipo numérico a estructura de fecha y estructura de fecha a string

Tomamos el número y convertimos

In [21]:
time.gmtime(t1 + 60 * 60 * 24 * 5)

time.struct_time(tm_year=2018, tm_mon=7, tm_mday=31, tm_hour=1, tm_min=36, tm_sec=2, tm_wday=1, tm_yday=212, tm_isdst=0)

Convertimos la estructura de tiempo en cadena

In [22]:
time.strftime("%Y-%m-%d %H:%M:%S", time.gmtime(t1 + 60 * 60 * 24 * 5))

'2018-07-31 01:36:02'

# Living code

In [31]:
pathYale = "C:/Users/Ian/Documents/PythonDataProducts4PredictiveAnalytics/BasicDataProcessing&Visualization/Final_Course1/datasets/yelp_academic_dataset_review.json"

fileYale = open(pathYale, 'r', encoding='UTF8')

In [32]:
datasetYale = []
for i in range(50000):
    datasetYale.append(json.loads(fileYale.readline()))
    
pprint.pprint(datasetYale[0])

{'business_id': '-MhfebM0QIsKt87iDN-FNw',
 'cool': 0,
 'date': '2015-04-15 05:21:16',
 'funny': 0,
 'review_id': 'xQY8N_XvtGbearJ5X4QryQ',
 'stars': 2.0,
 'text': 'As someone who has worked with many museums, I was eager to visit '
         'this gallery on my most recent trip to Las Vegas. When I saw they '
         'would be showing infamous eggs of the House of Faberge from the '
         'Virginia Museum of Fine Arts (VMFA), I knew I had to go!\n'
         '\n'
         'Tucked away near the gelateria and the garden, the Gallery is pretty '
         "much hidden from view. It's what real estate agents would call "
         '"cozy" or "charming" - basically any euphemism for small.\n'
         '\n'
         'That being said, you can still see wonderful art at a gallery of any '
         'size, so why the two *s you ask? Let me tell you:\n'
         '\n'
         '* pricing for this, while relatively inexpensive for a Las Vegas '
         'attraction, is completely over the top. For 

In [26]:
timeString = datasetYale[0]['date']
print(timeString)
print(type(timeString))

2015-04-15 05:21:16
<class 'str'>


In [41]:
timeStruct = time.strptime(timeString, "%Y-%m-%d %H:%M:%S")
timeStruct

time.struct_time(tm_year=2015, tm_mon=4, tm_mday=15, tm_hour=5, tm_min=21, tm_sec=16, tm_wday=2, tm_yday=105, tm_isdst=-1)

In [42]:
time.strptime("21:36:18, 28/05/2019", "%H:%M:%S, %d/%m/%Y")

time.struct_time(tm_year=2019, tm_mon=5, tm_mday=28, tm_hour=21, tm_min=36, tm_sec=18, tm_wday=1, tm_yday=148, tm_isdst=-1)

In [43]:
timeInt = time.mktime(timeStruct)
print(timeInt)
timeInt2 = time.mktime(time.strptime(datasetYale[99]['date'], "%Y-%m-%d %H:%M:%S"))
print(timeInt2)

1429093276.0
1457334948.0


In [44]:
timeDiff = timeInt2 - timeInt
print(timeDiff)

28241672.0


In [45]:
timeDiff / 60

470694.5333333333

In [46]:
timeDiff /(60*60)

7844.908888888889

In [47]:
timeDiff /(60*60* 24)

326.8712037037037